In [5]:
import pandas as pd
import pubchempy as pcp
from tqdm import tqdm
import numpy as np

In [6]:
df=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)

In [7]:
df.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [8]:
df.shape

(1048575, 18)

In [9]:
df.drug_row.nunique()

4510

In [10]:
df.drug_col.nunique()

582

In [11]:
df[df['drug_row'].isna() | df['drug_col'].isna()]

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
466033,466260,NVP-AEW541,NaN,1321N1,uM,NaN,7.488,0.0,0.0,0.0,0.0,8.000000,0.0,7.488,0.0,0.0,0.0,0.0
466034,466261,NVP-AEW541,NaN,22Rv1,uM,NaN,20.325,0.0,0.0,0.0,0.0,1.668671,0.0,20.325,0.0,0.0,0.0,0.0
466035,466262,NVP-AEW541,NaN,42-MG-BA,uM,NaN,8.884,0.0,0.0,0.0,0.0,1.548388,0.0,8.884,0.0,0.0,0.0,0.0
466036,466263,NVP-AEW541,NaN,5637,uM,NaN,9.584,0.0,0.0,0.0,0.0,8.000000,0.0,9.584,0.0,0.0,0.0,0.0
466037,466264,NVP-AEW541,NaN,639V,uM,NaN,18.442,0.0,0.0,0.0,0.0,6.695320,0.0,18.442,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048850,navitoclax:sorafenib (1:1 mol/mol),NaN,Mino,uM,NaN,62.810,0.0,0.0,0.0,0.0,0.043632,0.0,62.810,0.0,0.0,0.0,0.0
1048571,1048851,navitoclax:piperlongumine (1:1 mol/mol),NaN,Mino,uM,NaN,70.827,0.0,0.0,0.0,0.0,0.015539,0.0,70.827,0.0,0.0,0.0,0.0
1048572,1048852,67200-34-4,NaN,Mino,uM,NaN,43.276,0.0,0.0,0.0,0.0,0.513485,0.0,43.276,0.0,0.0,0.0,0.0
1048573,1048853,NVP-ADW742,NaN,Mino,uM,NaN,12.850,0.0,0.0,0.0,0.0,4.145955,0.0,12.850,0.0,0.0,0.0,0.0


In [12]:
df.dropna(inplace=True)

In [13]:
df.shape

(466033, 18)

In [14]:
len(np.unique(np.concatenate([df.drug_row.unique(),df.drug_col.unique()])))

4150

In [15]:

unique_drugs = pd.unique(df[['drug_row', 'drug_col']].values.ravel())

drug_df = pd.DataFrame(unique_drugs, columns=['drug'])

drug_df.head()


,drug
0,5-FU
1,ABT-888
2,AZD1775
3,BEZ-235
4,BORTEZOMIB


In [16]:
drug_df.shape

(4150, 1)

In [18]:
def get_smiles_from_name(name):
    compounds = pcp.get_compounds(name, 'name')
    if compounds:
        return compounds[0].isomeric_smiles
    return None

In [21]:
import requests

def get_smiles_from_name(name):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/IsomericSMILES/TXT'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        return response.text.strip()
    except Exception as e:
        print(f"Error retrieving SMILES for {name}: {e}")
        return None


In [22]:
tqdm.pandas()
drug_df['smiles_drug_a'] = drug_df['drug'].progress_apply(get_smiles_from_name)

  2%|█▌                                                                              | 81/4150 [01:04<51:27,  1.32it/s]

Error retrieving SMILES for (+/-)-SULFINPYRAZONE: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(+/-)-SULFINPYRAZONE/property/IsomericSMILES/TXT


 10%|███████▌                                                                       | 395/4150 [05:18<48:32,  1.29it/s]

Error retrieving SMILES for (6-)E-<U+200B>AMINOCAPROIC ACID: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/property/IsomericSMILES/TXT


 14%|███████████                                                                    | 584/4150 [07:56<48:18,  1.23it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████▌| 4129/4150 [55:44<00:16,  1.28it/s]

Error retrieving SMILES for AC1Q4OOX: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/AC1Q4OOX/property/IsomericSMILES/TXT


100%|█████████████████████████████████████████████████████████████████████████████▊| 4142/4150 [55:55<00:06,  1.30it/s]

Error retrieving SMILES for Vincristine Sulfate, Apocynaceae sp.: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Vincristine%20Sulfate,%20Apocynaceae%20sp./property/IsomericSMILES/TXT


100%|██████████████████████████████████████████████████████████████████████████████| 4150/4150 [56:02<00:00,  1.23it/s]


In [24]:
drug_df

,drug,smiles_drug_a
0,5-FU,C1=C(C(=O)NC(=O)N1)F
1,ABT-888,C[C@@]1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N
2,AZD1775,CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)N...
3,BEZ-235,CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N...
4,BORTEZOMIB,B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)...
...,...,...
4145,JNK-IN-7,CN(C)C/C=C/C(=O)NC1=CC=CC(=C1)C(=O)NC2=CC=C(C=...
4146,RA-XII,C[C@@H]1C(=O)N[C@H](C(=O)N([C@H](C(=O)N[C@H](C...
4147,vinorelbine,CCC1=C[C@H]2C[C@@](C3=C(CN(C2)C1)C4=CC=CC=C4N3...
4148,153381-68-1,CC1=C2[C@H](C(=O)[C@@]3([C@@H](C[C@@H]4[C@]([C...
